# lab 1 deteccion de pishing
- Marco Jurado 20308
- Oscar Lopez 20679

## Parte 1 – Ingeniería de características
### Exploración de datos

In [127]:
import pandas as pd
from pandas_profiling import ProfileReport
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, precision_score, recall_score, roc_curve, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC


In [128]:
data_set_original = pd.read_csv('dataset_pishing.csv')
data_set_original

,url,status
0,http://www.crestonwood.com/router.php,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,phishing
2,https://support-appleld.com.secureupdate.duila...,phishing
3,http://rgipt.ac.in,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,legitimate
...,...,...
11425,http://www.fontspace.com/category/blackletter,legitimate
11426,http://www.budgetbots.com/server.php/Server%20...,phishing
11427,https://www.facebook.com/Interactive-Televisio...,legitimate
11428,http://www.mypublicdomainpictures.com/,legitimate


In [129]:
data_set_original['status'].value_counts()

legitimate    5715
phishing      5715
Name: status, dtype: int64

Podemos observar que el dataset si se encuentra balanceado teniendo exactamente 5715 elementos para legitimate y la misma parte para phishing.

### Preprocesamiento

In [130]:
data_set_original['status'] = data_set_original['status'].map({'legitimate' : 0, 'phishing' : 1})
data_set_original

,url,status
0,http://www.crestonwood.com/router.php,0
1,http://shadetreetechnology.com/V4/validation/a...,1
2,https://support-appleld.com.secureupdate.duila...,1
3,http://rgipt.ac.in,0
4,http://www.iracing.com/tracks/gateway-motorspo...,0
...,...,...
11425,http://www.fontspace.com/category/blackletter,0
11426,http://www.budgetbots.com/server.php/Server%20...,1
11427,https://www.facebook.com/Interactive-Televisio...,0
11428,http://www.mypublicdomainpictures.com/,0


In [131]:
# Protocolo
data_set_original['protocol'] = data_set_original['url'].apply(lambda x: 1 if x.startswith('http://') else 0)
data_set_original['protocol'].value_counts()

1    6983
0    4447
Name: protocol, dtype: int64

In [132]:
def check_tld(url):
    accepted_tlds = ['com', 'net', 'org', 'edu', 'gov']
    domain_parts = url.split("//")[-1].split("/")[0]
    found_tld = any(domain_part for domain_part in domain_parts.split('.') if domain_part in accepted_tlds)
    return 0 if found_tld else 1

# Aplicar la función corregida
data_set_original['tld_check'] = data_set_original['url'].apply(check_tld)
data_set_original


,url,status,protocol,tld_check
0,http://www.crestonwood.com/router.php,0,1,0
1,http://shadetreetechnology.com/V4/validation/a...,1,1,0
2,https://support-appleld.com.secureupdate.duila...,1,0,0
3,http://rgipt.ac.in,0,1,1
4,http://www.iracing.com/tracks/gateway-motorspo...,0,1,0
...,...,...,...,...
11425,http://www.fontspace.com/category/blackletter,0,1,0
11426,http://www.budgetbots.com/server.php/Server%20...,1,1,0
11427,https://www.facebook.com/Interactive-Televisio...,0,0,0
11428,http://www.mypublicdomainpictures.com/,0,1,0


In [133]:
def tld_repetition(url):
    accepted_tlds = ['.com', '.net', '.org', '.edu', '.gov']
    tld_counts = sum(url.count(tld) for tld in accepted_tlds)
    return 1 if tld_counts > 1 else 0

# Aplicar
data_set_original['tld_repetition'] = data_set_original['url'].apply(tld_repetition)
data_set_original

,url,status,protocol,tld_check,tld_repetition
0,http://www.crestonwood.com/router.php,0,1,0,0
1,http://shadetreetechnology.com/V4/validation/a...,1,1,0,0
2,https://support-appleld.com.secureupdate.duila...,1,0,0,1
3,http://rgipt.ac.in,0,1,1,0
4,http://www.iracing.com/tracks/gateway-motorspo...,0,1,0,0
...,...,...,...,...,...
11425,http://www.fontspace.com/category/blackletter,0,1,0,0
11426,http://www.budgetbots.com/server.php/Server%20...,1,1,0,1
11427,https://www.facebook.com/Interactive-Televisio...,0,0,0,0
11428,http://www.mypublicdomainpictures.com/,0,1,0,0


In [134]:
def long_sld(url):
    domain = url.split("//")[-1].split("/")[0].split('?')[0]
    sld = domain.split('.')[-2] if len(domain.split('.')) > 1 else domain
    threshold = 10
    return 1 if len(sld) > threshold else 0

# Aplicar
data_set_original['long_sld'] = data_set_original['url'].apply(long_sld)
data_set_original

,url,status,protocol,tld_check,tld_repetition,long_sld
0,http://www.crestonwood.com/router.php,0,1,0,0,1
1,http://shadetreetechnology.com/V4/validation/a...,1,1,0,0,1
2,https://support-appleld.com.secureupdate.duila...,1,0,0,1,1
3,http://rgipt.ac.in,0,1,1,0,0
4,http://www.iracing.com/tracks/gateway-motorspo...,0,1,0,0,0
...,...,...,...,...,...,...
11425,http://www.fontspace.com/category/blackletter,0,1,0,0,0
11426,http://www.budgetbots.com/server.php/Server%20...,1,1,0,1,0
11427,https://www.facebook.com/Interactive-Televisio...,0,0,0,0,0
11428,http://www.mypublicdomainpictures.com/,0,1,0,0,1


In [135]:
def long_path(url):
    parts = url.split('//')
    path = parts[1].split('/', 1)[1] if len(parts) > 1 and '/' in parts[1] else ''
    return 1 if len(path) > 10 else 0

data_set_original['long_path'] = data_set_original['url'].apply(long_path)
data_set_original


,url,status,protocol,tld_check,tld_repetition,long_sld,long_path
0,http://www.crestonwood.com/router.php,0,1,0,0,1,0
1,http://shadetreetechnology.com/V4/validation/a...,1,1,0,0,1,1
2,https://support-appleld.com.secureupdate.duila...,1,0,0,1,1,1
3,http://rgipt.ac.in,0,1,1,0,0,0
4,http://www.iracing.com/tracks/gateway-motorspo...,0,1,0,0,0,1
...,...,...,...,...,...,...,...
11425,http://www.fontspace.com/category/blackletter,0,1,0,0,0,1
11426,http://www.budgetbots.com/server.php/Server%20...,1,1,0,1,0,1
11427,https://www.facebook.com/Interactive-Televisio...,0,0,0,0,0,1
11428,http://www.mypublicdomainpictures.com/,0,1,0,0,1,0


In [136]:
def check_chars_after_tld(url):
    match = re.search(r'https?://([^/]+)/?(.*)', url)
    if match:
        after_tld = match.group(2)
        # Buscar caracteres que no sean alfanuméricos, guiones, puntos o guiones bajos
        if re.search(r'[^a-zA-Z0-9\-._]', after_tld):
            return 1
    return 0

data_set_original['special_chars_after_tld'] = data_set_original['url'].apply(check_chars_after_tld)
data_set_original

,url,status,protocol,tld_check,tld_repetition,long_sld,long_path,special_chars_after_tld
0,http://www.crestonwood.com/router.php,0,1,0,0,1,0,0
1,http://shadetreetechnology.com/V4/validation/a...,1,1,0,0,1,1,1
2,https://support-appleld.com.secureupdate.duila...,1,0,0,1,1,1,1
3,http://rgipt.ac.in,0,1,1,0,0,0,0
4,http://www.iracing.com/tracks/gateway-motorspo...,0,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...
11425,http://www.fontspace.com/category/blackletter,0,1,0,0,0,1,1
11426,http://www.budgetbots.com/server.php/Server%20...,1,1,0,1,0,1,1
11427,https://www.facebook.com/Interactive-Televisio...,0,0,0,0,0,1,1
11428,http://www.mypublicdomainpictures.com/,0,1,0,0,1,0,0


In [137]:
data_set_original = data_set_original.drop(columns=['url'])
data_set_original

,status,protocol,tld_check,tld_repetition,long_sld,long_path,special_chars_after_tld
0,0,1,0,0,1,0,0
1,1,1,0,0,1,1,1
2,1,0,0,1,1,1,1
3,0,1,1,0,0,0,0
4,0,1,0,0,0,1,1
...,...,...,...,...,...,...,...
11425,0,1,0,0,0,1,1
11426,1,1,0,1,0,1,1
11427,0,0,0,0,0,1,1
11428,0,1,0,0,1,0,0


### Visualización de resultados

In [138]:
profile = ProfileReport(data_set_original, title="Analisis de Modelo de Deteccion de Phishing", explorative=True)
profile.to_file("analisis_modelo_phishing.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 509.02it/s]


### Selección de Características
Al observar podemos ver que special_chars_after_tld tiene mucha correlacion con long_path por lo tanto es probable que también se encuentren caracteres especiales a medida que haya un path largo por lo tanto sera eliminada.

In [139]:
data_set_original = data_set_original.drop(columns=['special_chars_after_tld'])
data_set_original

,status,protocol,tld_check,tld_repetition,long_sld,long_path
0,0,1,0,0,1,0
1,1,1,0,0,1,1
2,1,0,0,1,1,1
3,0,1,1,0,0,0
4,0,1,0,0,0,1
...,...,...,...,...,...,...
11425,0,1,0,0,0,1
11426,1,1,0,1,0,1
11427,0,0,0,0,0,1
11428,0,1,0,0,1,0


## Parte 2 – Implementación
### Separación de datos

In [140]:
train_df, temp_df = train_test_split(data_set_original, test_size=0.45, random_state=42)
validation_df, test_df = train_test_split(temp_df, test_size=2/3, random_state=42)

train_file_path = 'train_dataset.csv'
validation_file_path = 'validation_dataset.csv'
test_file_path = 'test_dataset.csv'

train_df.to_csv(train_file_path, index=False)
validation_df.to_csv(validation_file_path, index=False)
test_df.to_csv(test_file_path, index=False)

train_file_path, validation_file_path, test_file_path

('train_dataset.csv', 'validation_dataset.csv', 'test_dataset.csv')

### Implementación

In [141]:
train_df = pd.read_csv('train_dataset.csv')
validation_df = pd.read_csv('validation_dataset.csv')
test_df = pd.read_csv('test_dataset.csv')

X_train = train_df.drop('status', axis=1)
y_train = train_df['status']
X_validation = validation_df.drop('status', axis=1)
y_validation = validation_df['status']
X_test = test_df.drop('status', axis=1)
y_test = test_df['status']

In [142]:
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

lr_model = LogisticRegression(max_iter=1000, random_state=42)
lr_model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [143]:
def evaluate_model(model, X, y, name):
    y_pred = model.predict(X)
    y_proba = model.predict_proba(X)[:, 1]

    # Métricas
    conf_mat = confusion_matrix(y, y_pred)
    precision = precision_score(y, y_pred)
    recall = recall_score(y, y_pred)
    auc_score = roc_auc_score(y, y_proba)
    fpr, tpr, _ = roc_curve(y, y_proba)

    # Mostrar resultados
    print(f'{name} - Matriz de Confusión:\n{conf_mat}')
    print(f'{name} - Precisión: {precision}')
    print(f'{name} - Recall: {recall}')
    print(f'{name} - AUC: {auc_score}')

    # Curva ROC
    plt.figure()
    plt.plot(fpr, tpr, label=f'{name} (AUC = {auc_score})')
    plt.plot([0, 1], [0, 1], 'r--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'Curva ROC - {name}')
    plt.legend(loc='lower right')
    plt.show()

In [144]:
evaluate_model(rf_model, X_validation, y_validation, 'Random Forest - Validación')
evaluate_model(lr_model, X_validation, y_validation, 'Regresión Logística - Validación')

evaluate_model(rf_model, X_test, y_test, 'Random Forest - Prueba')
evaluate_model(lr_model, X_test, y_test, 'Regresión Logística - Prueba')

Random Forest - Validación - Matriz de Confusión:
[[652 200]
 [425 437]]
Random Forest - Validación - Precisión: 0.6860282574568289
Random Forest - Validación - Recall: 0.5069605568445475
Random Forest - Validación - AUC: 0.6916093428319335
Regresión Logística - Validación - Matriz de Confusión:
[[610 242]
 [406 456]]
Regresión Logística - Validación - Precisión: 0.6532951289398281
Regresión Logística - Validación - Recall: 0.5290023201856149
Regresión Logística - Validación - AUC: 0.6520136869165496
Random Forest - Prueba - Matriz de Confusión:
[[1332  383]
 [ 859  856]]
Random Forest - Prueba - Precisión: 0.6908797417271993
Random Forest - Prueba - Recall: 0.49912536443148686
Random Forest - Prueba - AUC: 0.6897695688021147


C:\Users\marco\AppData\Local\Temp\ipykernel_32272\2406058967.py:26: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()
C:\Users\marco\AppData\Local\Temp\ipykernel_32272\2406058967.py:26: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()
C:\Users\marco\AppData\Local\Temp\ipykernel_32272\2406058967.py:26: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


Regresión Logística - Prueba - Matriz de Confusión:
[[1270  445]
 [ 822  893]]
Regresión Logística - Prueba - Precisión: 0.6674140508221226
Regresión Logística - Prueba - Recall: 0.5206997084548105
Regresión Logística - Prueba - AUC: 0.660565750665114


C:\Users\marco\AppData\Local\Temp\ipykernel_32272\2406058967.py:26: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [145]:
gb_model = GradientBoostingClassifier(random_state=42)
gb_model.fit(X_train, y_train)

evaluate_model(gb_model, X_validation, y_validation, 'Gradient Boosting - Validación')
evaluate_model(gb_model, X_test, y_test, 'Gradient Boosting - Prueba')

Gradient Boosting - Validación - Matriz de Confusión:
[[652 200]
 [425 437]]
Gradient Boosting - Validación - Precisión: 0.6860282574568289
Gradient Boosting - Validación - Recall: 0.5069605568445475
Gradient Boosting - Validación - AUC: 0.6929872934435694
Gradient Boosting - Prueba - Matriz de Confusión:
[[1332  383]
 [ 859  856]]
Gradient Boosting - Prueba - Precisión: 0.6908797417271993
Gradient Boosting - Prueba - Recall: 0.49912536443148686
Gradient Boosting - Prueba - AUC: 0.6897654888694337


C:\Users\marco\AppData\Local\Temp\ipykernel_32272\2406058967.py:26: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()
C:\Users\marco\AppData\Local\Temp\ipykernel_32272\2406058967.py:26: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [146]:
svm_model = SVC(probability=True, kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)

# Evaluar el modelo en el conjunto de validación y prueba
evaluate_model(svm_model, X_validation, y_validation, 'SVM - Validación')
evaluate_model(svm_model, X_test, y_test, 'SVM - Prueba')

SVM - Validación - Matriz de Confusión:
[[676 176]
 [501 361]]
SVM - Validación - Precisión: 0.6722532588454376
SVM - Validación - Recall: 0.41879350348027844
SVM - Validación - AUC: 0.6146109059616788


C:\Users\marco\AppData\Local\Temp\ipykernel_32272\2406058967.py:26: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


SVM - Prueba - Matriz de Confusión:
[[1370  345]
 [1004  711]]
SVM - Prueba - Precisión: 0.6732954545454546
SVM - Prueba - Recall: 0.4145772594752187
SVM - Prueba - AUC: 0.6123805557208305


C:\Users\marco\AppData\Local\Temp\ipykernel_32272\2406058967.py:26: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


# Preguntas

1. ¿Qué ventajas tiene el análisis de una URL contra el análisis de otros datos, cómo el tiempo
de vida del dominio, o las características de la página Web?

R// En este caso el análisis de la URL agiliza el proceso de conocimiento si un sitio al que se puede ingresar es Phishing o probable de serlo. Esto debido a que para realizar este análisis se pueden usar modelod y características previamente conocidas sobre los sitios peligrosos. Esto llevando a evitar realizar un análisis mayormente exhaustivo y logrando así la detección temprana del sitio.

2. ¿Qué características de una URL son más prometedoras para la detección de phishing?

R// En este caso para el análisis de la URL las características mayormente importantes para lograr conocer y realizar la detección del Phishing, pueden ser:
- La longitud que puede tener la URL
- El dominio que puede tener esta URL
- Los caracteres que pueden estar involucrados en la URL
- El protocolo que tiene la URL

3. ¿Cuál es el impacto de clasificar un sitio legítimo como phishing?

R// Al momento de tener esta acción el impacto sobre la página que se determina como phishing sería muy negativo, debido a que al momento de determinar el caso contrario al que es, esta página perdería la confianza de los usuarios a los cuáles podría llegar a cautivar. En el caso del sistema esto puede generar una pérdida de confianza para los usuarios que conozcan la realidad sobre el sitio que ha sido mal clasificado.

4. ¿Cuál es el impacto de clasificar un sitio de phishing como legítimo?

R// Principalmente esto presenta una falla de seguridad debido a que las personas que no tengan el conocimiento y confien en el sistema pueden llegar a ser víctimas de la ciberdelincuencia que pueden brindar estos sitios maliciosos, debido a esto muchas personas pueden perder la completa confianza en el sistema e indicar que no sirve de manera correcta.

5. En base a las respuestas anteriores, ¿Qué métrica elegiría para comparar modelos similares
de clasificación de phishing?

R// En este caso la mejor métrica para realizar la comparación de los modelos puede ser la precisión debido a que los modelos deben de ser lo más precisos posibles ya que cualquier falla entre ellos puede causar repercusiones grandes en el uso que puede tener cada uno de estos. Otra muy importante puede ser el recall debido a que esta pueden indicar la cantidad de verdaderos sitios que son phishing con respecto a los sitios existentes. Por esto estas dos métricas son muy importantes para conocer el mejor modelo de clasificación.

6. ¿Qué modelo funcionó mejor para la clasificación de phishing? ¿Por qué?

R// En este caso realizando la comparación entre las distintas métricas de desempeño que tienen los modelos probados con la información se puede conocer que el modelo de Gradient Boosting fue el que mejor rendimiento tuvo a comparación de todos los otros, debido a la cantidad más alta a comparación de las métricas de prueba de los otros modelos.

7. Una empresa desea utilizar su mejor modelo, debido a que sus empleados sufren constantes
ataques de phishing mediante e-mail. La empresa estima que, de un total de 50,000 emails,
un 15% son phishing. ¿Qué cantidad de alarmas generaría su modelo? ¿Cuántas positivas y
cuantas negativas? ¿Funciona el modelo para el BR propuesto? En caso negativo, ¿qué se
podría hacer para reducir la cantidad de falsas alarmas?